In [1]:
source("https://bioconductor.org/biocLite.R")
biocLite("DESeq2")
biocLite('preprocessCore')

Bioconductor version 3.6 (BiocInstaller 1.28.0), ?biocLite for help
A new version of Bioconductor is available after installing the most recent
  version of R; see http://bioconductor.org/install
BioC_mirror: https://bioconductor.org
Using Bioconductor 3.6 (BiocInstaller 1.28.0), R 3.4.1 (2017-06-30).
Installing package(s) ‘DESeq2’
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'bindrcpp', 'bit', 'bit64', 'broom', 'caret', 'caTools',
  'config', 'CVST', 'dbplyr', 'ddalpha', 'devtools', 'dplyr', 'evaluate',
  'forecast', 'foreign', 'ggplot2', 'git2r', 'glue', 'haven', 'highr', 'hms',
  'htmlwidgets', 'httpuv', 'IRdisplay', 'iterators', 'kernlab', 'lattice',
  'lava', 'lmtest', 'magic', 'MASS', 'Matrix', 'matrixStats', 'modelr',
  'munsell', 'nlme', 'nycflights13', 'pbdZMQ', 'pillar', 'plogr', 'psych',
  'purrr', 'randomForest', 'Rcpp', 'RcppArmadillo', 'RcppRoll', 'RCurl',
  'recipes', 'repr', 'rlang', 'rmarkdown', 'robustbase', 'RSQLite', 's

In [2]:
biocLite('preprocessCore')

BioC_mirror: https://bioconductor.org
Using Bioconductor 3.6 (BiocInstaller 1.28.0), R 3.4.1 (2017-06-30).
Installing package(s) ‘preprocessCore’
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Old packages: 'bindrcpp', 'bit', 'bit64', 'broom', 'caret', 'caTools',
  'config', 'CVST', 'dbplyr', 'ddalpha', 'devtools', 'dplyr', 'evaluate',
  'forecast', 'foreign', 'ggplot2', 'git2r', 'glue', 'haven', 'highr', 'hms',
  'htmlwidgets', 'httpuv', 'IRdisplay', 'iterators', 'kernlab', 'lattice',
  'lava', 'lmtest', 'magic', 'MASS', 'Matrix', 'matrixStats', 'modelr',
  'munsell', 'nlme', 'nycflights13', 'pbdZMQ', 'pillar', 'plogr', 'psych',
  'purrr', 'randomForest', 'Rcpp', 'RcppArmadillo', 'RcppRoll', 'RCurl',
  'recipes', 'repr', 'rlang', 'rmarkdown', 'robustbase', 'RSQLite', 'selectr',
  'sfsmisc', 'shiny', 'sourcetools', 'sparklyr', 'splitstackshape', 'stringi',
  'stringr', 'survival', 'tidyr', 'tidyverse', 'timeDate', 'tseries', 'utf8',
  'withr', 'XML', 'xml

In [3]:
require(DESeq2)
require(preprocessCore)
require(readr)

Loading required package: DESeq2
Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which, which.

In [4]:
featureCount <- as.data.frame(read_tsv("../data/pos_and_neg_by_score.tsv"))

row.names(featureCount) <- featureCount$X1

featureCount$X1 <- NULL

her2.clinical <- featureCount[, c('her2_status_by_ihc', 'her2_ihc_score')]
featureCount <- subset(featureCount, select = -c(her2_status_by_ihc, her2_ihc_score))

#row.names(her2.clinical) <- her2.clinical$X1

her2.clinical$X1 <- NULL
her2.clinical$X2 <- NULL
her2.clinical$X3 <- NULL

#her2.clinical <- as.data.frame(t(her2.clinical))
#her2.clinical <- subset(her2.clinical, her2_status_by_ihc == "Positive" | her2_status_by_ihc == "Negative")
her2.clinical <- her2.clinical[order(row.names(her2.clinical)), ]
featureCount <- featureCount[order(row.names(featureCount)), ]
featureCount <- as.data.frame(t(featureCount))

head(her2.clinical)

#featureCount <- featureCount[row.names(her2.clinical)]

Warning message:
“Missing column names filled in: 'X1' [1]”Parsed with column specification:
cols(
  .default = col_integer(),
  X1 = col_character(),
  her2_status_by_ihc = col_character(),
  her2_ihc_score = col_character()
)
See spec(...) for full column specifications.


,her2_status_by_ihc,her2_ihc_score
TCGA-A1-A0SK-01A-12R-A084-07,Negative,0
TCGA-A1-A0SM-01A-11R-A084-07,Positive,3+
TCGA-A1-A0SP-01A-11R-A084-07,Negative,0
TCGA-A2-A04X-01A-21R-A034-07,Positive,3+
TCGA-A2-A0CM-01A-31R-A034-07,Negative,0
TCGA-A2-A0CX-01A-21R-A00Z-07,Positive,3+


In [5]:
featureCount <- as.matrix(featureCount)

head(featureCount)
str(featureCount)

dsq.fc <- DESeqDataSetFromMatrix(countData = featureCount, colData = her2.clinical, design = ~ her2_status_by_ihc)

vsd.fc <- vst(dsq.fc)

head(as.data.frame(assay(vsd.fc)))

vsd.data <- as.data.frame(t(assay(vsd.fc)))

,TCGA-A1-A0SK-01A-12R-A084-07,TCGA-A1-A0SM-01A-11R-A084-07,TCGA-A1-A0SP-01A-11R-A084-07,TCGA-A2-A04X-01A-21R-A034-07,TCGA-A2-A0CM-01A-31R-A034-07,TCGA-A2-A0CX-01A-21R-A00Z-07,TCGA-A2-A0D0-01A-11R-A00Z-07,TCGA-A2-A0D1-01A-11R-A034-07,TCGA-A2-A0D2-01A-21R-A034-07,TCGA-A2-A0EQ-01A-11R-A034-07,⋯,TCGA-EW-A1IW-01A-11R-A13Q-07,TCGA-EW-A1J3-01A-11R-A13Q-07,TCGA-EW-A6S9-01A-22R-A33J-07,TCGA-JL-A3YX-01A-11R-A22U-07,TCGA-LD-A9QF-01A-32R-A41B-07,TCGA-LL-A5YL-01A-12R-A29R-07,TCGA-LL-A7T0-01A-31R-A352-07,TCGA-LL-A9Q3-01A-11R-A41B-07,TCGA-S3-AA14-01A-11R-A41B-07,TCGA-UU-A93S-01A-21R-A41B-07
1/2-SBSRNA4,20,22,35,58,20,23,20,66,45,25,⋯,39,96,43,62,29,65,22,32,18,13
A1BG,295,1051,106,123,113,202,50,262,63,64,⋯,261,312,319,1119,313,1532,504,1288,197,449
A1BG-AS1,47,124,32,47,52,57,8,98,21,28,⋯,54,95,139,176,56,69,61,58,58,43
A1CF,3,13,3,2,2,3,3,15,2,19,⋯,5,49,15,1,4,3,16,1,23,0
A2LD1,1523,359,232,210,84,187,81,87,369,442,⋯,409,153,397,952,327,454,201,377,201,352
A2M,14162,30567,15544,28201,19431,42717,302874,15321,114300,29686,⋯,58031,33608,26189,21074,47049,21122,15866,16331,23987,5130


 int [1:23368, 1:145] 20 295 47 3 1523 14162 128 0 278 2 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:23368] "1/2-SBSRNA4" "A1BG" "A1BG-AS1" "A1CF" ...
  ..$ : chr [1:145] "TCGA-A1-A0SK-01A-12R-A084-07" "TCGA-A1-A0SM-01A-11R-A084-07" "TCGA-A1-A0SP-01A-11R-A084-07" "TCGA-A2-A04X-01A-21R-A034-07" ...


Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”

,TCGA-A1-A0SK-01A-12R-A084-07,TCGA-A1-A0SM-01A-11R-A084-07,TCGA-A1-A0SP-01A-11R-A084-07,TCGA-A2-A04X-01A-21R-A034-07,TCGA-A2-A0CM-01A-31R-A034-07,TCGA-A2-A0CX-01A-21R-A00Z-07,TCGA-A2-A0D0-01A-11R-A00Z-07,TCGA-A2-A0D1-01A-11R-A034-07,TCGA-A2-A0D2-01A-21R-A034-07,TCGA-A2-A0EQ-01A-11R-A034-07,⋯,TCGA-EW-A1IW-01A-11R-A13Q-07,TCGA-EW-A1J3-01A-11R-A13Q-07,TCGA-EW-A6S9-01A-22R-A33J-07,TCGA-JL-A3YX-01A-11R-A22U-07,TCGA-LD-A9QF-01A-32R-A41B-07,TCGA-LL-A5YL-01A-12R-A29R-07,TCGA-LL-A7T0-01A-31R-A352-07,TCGA-LL-A9Q3-01A-11R-A41B-07,TCGA-S3-AA14-01A-11R-A41B-07,TCGA-UU-A93S-01A-21R-A41B-07
1/2-SBSRNA4,6.082627,6.131936,6.448612,6.756771,6.249757,6.283915,6.270566,7.179743,6.680999,6.311792,⋯,6.307493,7.128255,6.594930,6.727766,6.513803,7.030415,6.341274,6.715544,6.307935,6.371852
A1BG,8.290642,9.904096,7.317543,7.401023,7.598588,8.137945,6.892012,8.653114,6.940544,6.970243,⋯,7.882812,8.344821,8.517983,9.892243,8.829320,10.777829,9.438256,10.969830,8.441849,10.053306
A1BG-AS1,6.577307,7.376288,6.392730,6.604385,6.893497,6.904004,5.847374,7.554811,6.203641,6.378950,⋯,6.508406,7.119064,7.595903,7.652154,7.007112,7.081538,7.080409,7.203281,7.162096,7.284204
A1CF,5.478216,5.896918,5.498194,5.412807,5.462680,5.529130,5.555418,6.149611,5.438400,6.161894,⋯,5.534440,6.599490,6.002711,5.305317,5.634581,5.533909,6.165187,5.388685,6.456823,5.085519
A2LD1,10.400985,8.528584,8.129836,7.951405,7.309824,8.051231,7.310758,7.437445,8.787805,9.039409,⋯,8.390505,7.567590,8.784335,9.676374,8.883831,9.145255,8.296888,9.307035,8.465728,9.726485
A2M,13.552803,14.665901,13.829814,14.598841,14.473086,15.493762,18.485238,14.273625,16.832986,14.919691,⋯,15.233832,14.781153,14.600733,14.028313,15.839006,14.510938,14.274779,14.588950,15.076138,13.482341


In [6]:
vsd.data <- cbind(vsd.data, her2.clinical)

write.table(vsd.data, file = '../data/vsd_posneg_score.tsv', quote = F, sep = '\t', col.names = NA)

In [7]:
as.data.frame(read_tsv("../data/vsd_posneg_score.tsv"))

Warning message:
“Missing column names filled in: 'X1' [1]”Parsed with column specification:
cols(
  .default = col_double(),
  X1 = col_character(),
  her2_status_by_ihc = col_character(),
  her2_ihc_score = col_character()
)
See spec(...) for full column specifications.
ERROR while rich displaying an object: Error in sprintf(wrap, header, body): 'fmt' length exceeds maximal format length 8192

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     rpr <- mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     rpr <- mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. mime2repr

X1,1/2-SBSRNA4,A1BG,A1BG-AS1,A1CF,A2LD1,A2M,A2ML1,A2MP1,A4GALT,⋯,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,tAKR,her2_status_by_ihc,her2_ihc_score
TCGA-A1-A0SK-01A-12R-A084-07,6.082627,8.290642,6.577307,5.478216,10.400985,13.55280,7.401659,5.085519,8.222130,⋯,10.644123,12.32694,9.179333,11.96196,10.79019,12.34998,11.92737,5.682949,Negative,0
TCGA-A1-A0SM-01A-11R-A084-07,6.131936,9.904096,7.376288,5.896918,8.528584,14.66590,5.725202,5.539680,9.958057,⋯,10.524111,11.54569,7.726173,12.01857,13.69622,11.76092,12.04806,5.313302,Positive,3+
TCGA-A1-A0SP-01A-11R-A084-07,6.448612,7.317543,6.392730,5.498194,8.129836,13.82981,11.305395,5.498194,11.133395,⋯,9.617126,11.51345,9.617126,10.92774,13.24631,11.01940,11.96628,5.755673,Negative,0
TCGA-A2-A04X-01A-21R-A034-07,6.756771,7.401023,6.604385,5.412807,7.951405,14.59884,6.782464,5.601364,8.312302,⋯,10.313625,11.50924,7.826013,10.93796,12.86268,11.53294,10.96355,5.085519,Positive,3+
TCGA-A2-A0CM-01A-31R-A034-07,6.249757,7.598588,6.893497,5.462680,7.309824,14.47309,11.486470,5.617408,8.899101,⋯,9.879387,11.60526,9.874008,10.86896,13.62999,11.57302,11.66248,5.462680,Negative,0
TCGA-A2-A0CX-01A-21R-A00Z-07,6.283915,8.137945,6.904004,5.529130,8.051231,15.49376,11.802891,5.597093,9.241476,⋯,10.219031,11.65197,10.733302,12.11394,13.66237,11.17957,10.86015,5.656739,Positive,3+
TCGA-A2-A0D0-01A-11R-A00Z-07,6.270566,6.892012,5.847374,5.555418,7.310758,18.48524,6.298237,6.826169,8.113443,⋯,9.713171,12.40328,10.200313,11.66881,14.75355,10.77797,11.58082,5.357613,Negative,0
TCGA-A2-A0D1-01A-11R-A034-07,7.179743,8.653114,7.554811,6.149611,7.437445,14.27362,6.333982,5.870633,8.196195,⋯,10.116839,11.05570,7.459562,11.20176,13.72570,11.50014,11.44052,5.366095,Positive,3+
TCGA-A2-A0D2-01A-21R-A034-07,6.680999,6.940544,6.203641,5.438400,8.787805,16.83299,5.827747,5.641419,8.856512,⋯,10.204067,11.45915,9.391282,11.81971,14.99571,11.19587,11.48610,5.517175,Negative,0
TCGA-A2-A0EQ-01A-11R-A034-07,6.311792,6.970243,6.378950,6.161894,9.039409,14.91969,7.069674,5.647033,8.679202,⋯,10.613122,10.86041,10.230541,11.15093,13.51873,11.58887,11.10662,5.647033,Positive,3+


In [8]:
nq.fc <- normalize.quantiles(featureCount)

colnames(nq.fc) <- colnames(featureCount)
row.names(nq.fc) <- row.names(featureCount)

nq.fc <- as.data.frame(t(nq.fc))

In [9]:
nq.data <- cbind(nq.fc, her2.clinical)

In [10]:
write.table(nq.data, "../data/norm_quan_posneg_score.tsv", sep = '\t', quote = F, col.names = NA)

In [11]:
featureCount <- as.data.frame(read_tsv("../data/GSM1536837_06_01_15_TCGA_24.tumor_Rsubread_FeatureCounts.txt.gz"))

row.names(featureCount) <- featureCount$X1

featureCount$X1 <- NULL

her2.clinical <- as.data.frame(read_tsv("../data/her2_Clinical_Data.tsv"))

row.names(her2.clinical) <- her2.clinical$X1

her2.clinical$X1 <- NULL
her2.clinical$X2 <- NULL
her2.clinical$X3 <- NULL

her2.clinical <- as.data.frame(t(her2.clinical))
her2.fish <- subset(her2.clinical, her2_fish_status == "Positive" | her2_fish_status == "Negative")
her2.fish <- her2.fish[order(row.names(her2.fish)), ]
her2.fish.score <- subset(her2.fish, (her2_status_by_ihc == "Positive" & her2_ihc_score == "3+") | (her2_status_by_ihc == "Negative" & her2_ihc_score == "0"))
her2.fish.copy <- subset()
her2.clinical <- subset(her2.clinical, her2_status_by_ihc == "Positive" | her2_status_by_ihc == "Negative")
her2.clinical <- her2.clinical[order(row.names(her2.clinical)), ]

head(her2.clinical)

fishCount <- featureCount[row.names(her2.fish)]
fishCountScore <- fishCount[row.names(her2.fish.score)]
featureCount <- featureCount[row.names(her2.clinical)]

Warning message:
“Missing column names filled in: 'X1' [1]”Parsed with column specification:
cols(
  .default = col_integer(),
  X1 = col_character()
)
See spec(...) for full column specifications.
Warning message:
“Missing column names filled in: 'X1' [1], 'X2' [2], 'X3' [3]”Parsed with column specification:
cols(
  .default = col_character()
)
See spec(...) for full column specifications.


,her2_status_by_ihc,her2_ihc_percent_positive,her2_ihc_score,her2_positivity_scale_other,her2_positivity_method_text,her2_fish_status,her2_copy_number,her2_and_cent17_cells_count,her2_cent17_ratio,her2_and_cent17_scale_other,⋯,nte_her2_status_ihc__positive,nte_her2_positivity_ihc_score,nte_her2_positivity_other_scale,nte_her2_positivity_method,nte_her2_fish_status,nte_her2_signal_number,her2_cent17_counted_cells_count,nte_cent_17_her2_ratio,nte_cent17_her2_other_scale,nte_her2_fish_define_method
TCGA-3C-AAAU-01A-11R-A41B-07,Negative,[Not Available],[Not Available],[Not Available],[Not Available],[Not Evaluated],[Not Available],[Not Available],[Not Available],[Not Available],⋯,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available]
TCGA-3C-AALI-01A-11R-A41B-07,Positive,[Not Available],[Not Available],[Not Available],[Not Available],[Not Evaluated],[Not Available],[Not Available],[Not Available],[Not Available],⋯,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available]
TCGA-3C-AALK-01A-11R-A41B-07,Positive,[Not Available],[Not Available],[Not Available],[Not Available],[Not Evaluated],[Not Available],[Not Available],[Not Available],[Not Available],⋯,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available]
TCGA-5L-AAT0-01A-12R-A41B-07,Negative,[Not Available],1+,[Not Available],[Not Available],[Not Evaluated],[Not Available],[Not Available],[Not Available],[Not Available],⋯,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available]
TCGA-A1-A0SB-01A-11R-A144-07,Negative,[Not Available],[Not Available],[Not Available],[Not Available],[Not Evaluated],[Not Available],[Not Available],[Not Available],[Not Available],⋯,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available]
TCGA-A1-A0SD-01A-11R-A115-07,Negative,[Not Available],[Not Available],[Not Available],[Not Available],[Not Evaluated],[Not Available],[Not Available],[Not Available],[Not Available],⋯,[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available],[Not Available]


In [12]:
featureCount <- as.matrix(featureCount)

head(featureCount)
str(featureCount)

dsq.fc <- DESeqDataSetFromMatrix(countData = featureCount, colData = her2.clinical, design = ~ her2_status_by_ihc)

vsd.fc <- vst(dsq.fc)

head(as.data.frame(assay(vsd.fc)))

vsd.data <- as.data.frame(t(assay(vsd.fc)))

,TCGA-3C-AAAU-01A-11R-A41B-07,TCGA-3C-AALI-01A-11R-A41B-07,TCGA-3C-AALK-01A-11R-A41B-07,TCGA-5L-AAT0-01A-12R-A41B-07,TCGA-A1-A0SB-01A-11R-A144-07,TCGA-A1-A0SD-01A-11R-A115-07,TCGA-A1-A0SE-01A-11R-A084-07,TCGA-A1-A0SF-01A-11R-A144-07,TCGA-A1-A0SG-01A-11R-A144-07,TCGA-A1-A0SI-01A-11R-A144-07,⋯,TCGA-S3-AA14-01A-11R-A41B-07,TCGA-S3-AA15-01A-11R-A41B-07,TCGA-UL-AAZ6-01A-11R-A41B-07,TCGA-UU-A93S-01A-21R-A41B-07,TCGA-W8-A86G-01A-21R-A36F-07,TCGA-WT-AB44-01A-11R-A41B-07,TCGA-XX-A899-01A-11R-A36F-07,TCGA-XX-A89A-01A-11R-A36F-07,TCGA-Z7-A8R5-01A-42R-A41B-07,TCGA-Z7-A8R6-01A-11R-A41B-07
1/2-SBSRNA4,107,49,36,27,50,35,27,33,43,35,⋯,18,23,53,13,92,77,57,34,42,42
A1BG,462,457,478,363,102,463,622,832,486,453,⋯,197,310,101,449,1103,553,553,537,822,749
A1BG-AS1,135,139,70,57,37,99,104,100,90,142,⋯,58,60,24,43,200,42,96,120,53,109
A1CF,1,17,4,0,7,4,1,3,0,4,⋯,23,5,25,0,4,4,3,26,0,8
A2LD1,363,153,164,213,107,241,261,425,373,229,⋯,201,241,522,352,142,65,405,199,147,77
A2M,17802,14734,27976,22176,78624,77611,51671,40094,37544,36101,⋯,23987,44626,12239,5130,41116,6909,55007,39127,64590,22219


 int [1:23368, 1:708] 107 462 135 1 363 17802 25 0 225 33 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:23368] "1/2-SBSRNA4" "A1BG" "A1BG-AS1" "A1CF" ...
  ..$ : chr [1:708] "TCGA-3C-AAAU-01A-11R-A41B-07" "TCGA-3C-AALI-01A-11R-A41B-07" "TCGA-3C-AALK-01A-11R-A41B-07" "TCGA-5L-AAT0-01A-12R-A41B-07" ...


factor levels were dropped which had no samples


,TCGA-3C-AAAU-01A-11R-A41B-07,TCGA-3C-AALI-01A-11R-A41B-07,TCGA-3C-AALK-01A-11R-A41B-07,TCGA-5L-AAT0-01A-12R-A41B-07,TCGA-A1-A0SB-01A-11R-A144-07,TCGA-A1-A0SD-01A-11R-A115-07,TCGA-A1-A0SE-01A-11R-A084-07,TCGA-A1-A0SF-01A-11R-A144-07,TCGA-A1-A0SG-01A-11R-A144-07,TCGA-A1-A0SI-01A-11R-A144-07,⋯,TCGA-S3-AA14-01A-11R-A41B-07,TCGA-S3-AA15-01A-11R-A41B-07,TCGA-UL-AAZ6-01A-11R-A41B-07,TCGA-UU-A93S-01A-21R-A41B-07,TCGA-W8-A86G-01A-21R-A36F-07,TCGA-WT-AB44-01A-11R-A41B-07,TCGA-XX-A899-01A-11R-A36F-07,TCGA-XX-A89A-01A-11R-A36F-07,TCGA-Z7-A8R5-01A-42R-A41B-07,TCGA-Z7-A8R6-01A-11R-A41B-07
1/2-SBSRNA4,7.306247,7.022133,6.568893,6.584545,6.864746,6.395115,6.242580,6.515852,6.623516,6.322369,⋯,6.320297,6.492039,6.859276,6.374782,7.435710,7.920301,6.849199,6.602599,6.940104,6.603812
A1BG,8.934789,9.518239,9.211639,9.256888,7.504025,8.870188,9.246233,9.955362,9.105042,8.694478,⋯,8.487000,9.084897,7.430301,10.083328,10.467818,10.407190,9.294902,9.500631,10.384601,9.663416
A1BG-AS1,7.531638,8.054794,7.088004,7.180981,6.636822,7.185275,7.230568,7.425346,7.223465,7.410418,⋯,7.191378,7.249526,6.312882,7.298866,8.267512,7.303920,7.300198,7.706571,7.139325,7.399373
A1CF,5.305056,6.273117,5.587326,5.066380,5.776495,5.529627,5.298809,5.520154,5.066380,5.502852,⋯,6.472510,5.752005,6.337057,5.066380,5.614538,5.819634,5.500275,6.423539,5.066380,5.762793
A2LD1,8.636072,8.163266,7.927666,8.588093,7.551848,8.091903,8.183051,9.072573,8.772186,7.902331,⋯,8.511084,8.768560,9.330929,9.755725,7.882976,7.738694,8.897565,8.264666,8.181806,7.080408
A2M,14.004034,14.398579,14.917947,15.030144,16.506534,16.046345,15.460913,15.449120,15.197961,14.770977,⋯,15.134716,16.072006,13.735151,13.515248,15.620845,13.984198,15.774643,15.553093,16.608356,14.436708


In [13]:
vsd.data <- cbind(vsd.data, her2.clinical)

write.table(vsd.data, file = '../data/vsd_FeatureCounts.tsv', quote = F, sep = '\t', col.names = NA)

In [ ]:
as.data.frame(read_tsv("../data/vsd_FeatureCounts.tsv"))

In [15]:
nq.fc <- normalize.quantiles(featureCount)

colnames(nq.fc) <- colnames(featureCount)
row.names(nq.fc) <- row.names(featureCount)

nq.fc <- as.data.frame(t(nq.fc))

In [16]:
nq.data <- cbind(nq.fc, her2.clinical)

In [17]:
write.table(nq.data, "../data/norm_quan_FeatureCount.tsv", sep = '\t', quote = F, col.names = NA)

In [18]:
fishCount <- as.matrix(fishCount)

head(fishCount)
str(fishCount)

dsq.fc <- DESeqDataSetFromMatrix(countData = fishCount, colData = her2.fish, design = ~ her2_fish_status)

vsd.fc <- vst(dsq.fc)

head(as.data.frame(assay(vsd.fc)))

vsd.data <- as.data.frame(t(assay(vsd.fc)))

,TCGA-5T-A9QA-01A-11R-A41B-07,TCGA-A1-A0SE-01A-11R-A084-07,TCGA-A1-A0SH-01A-11R-A084-07,TCGA-A1-A0SJ-01A-11R-A084-07,TCGA-A1-A0SM-01A-11R-A084-07,TCGA-A1-A0SO-01A-22R-A084-07,TCGA-A2-A04N-01A-11R-A115-07,TCGA-A2-A04P-01A-31R-A034-07,TCGA-A2-A04Q-01A-21R-A034-07,TCGA-A2-A04R-01A-41R-A109-07,⋯,TCGA-OL-A6VQ-01A-12R-A41B-07,TCGA-OL-A6VR-01A-32R-A33J-07,TCGA-OL-A97C-01A-32R-A41B-07,TCGA-PE-A5DD-01A-12R-A27Q-07,TCGA-S3-A6ZF-01A-32R-A32P-07,TCGA-S3-A6ZH-01A-22R-A32P-07,TCGA-S3-AA0Z-01A-11R-A41B-07,TCGA-S3-AA11-01A-31R-A41B-07,TCGA-S3-AA12-01A-11R-A41B-07,TCGA-S3-AA17-01A-11R-A41B-07
1/2-SBSRNA4,24,27,38,35,22,261,45,81,63,43,⋯,16,85,15,76,67,24,64,75,73,49
A1BG,626,622,556,697,1051,1173,535,225,301,644,⋯,278,1075,481,1275,901,592,277,289,328,483
A1BG-AS1,38,104,92,81,124,202,102,43,77,97,⋯,68,188,100,195,194,115,211,91,71,85
A1CF,33,1,3,3,13,5,4,10,2,10,⋯,0,3,1,3,8,8,12,4,0,4
A2LD1,504,261,415,423,359,441,390,175,339,230,⋯,98,838,109,166,399,408,182,251,215,206
A2M,6426,51671,41485,65022,30567,16537,28088,19462,36162,22707,⋯,18167,20699,23223,33323,25722,22562,26931,20451,6413,30266


 int [1:23368, 1:403] 24 626 38 33 504 6426 1 1 816 11 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:23368] "1/2-SBSRNA4" "A1BG" "A1BG-AS1" "A1CF" ...
  ..$ : chr [1:403] "TCGA-5T-A9QA-01A-11R-A41B-07" "TCGA-A1-A0SE-01A-11R-A084-07" "TCGA-A1-A0SH-01A-11R-A084-07" "TCGA-A1-A0SJ-01A-11R-A084-07" ...


factor levels were dropped which had no samples


,TCGA-5T-A9QA-01A-11R-A41B-07,TCGA-A1-A0SE-01A-11R-A084-07,TCGA-A1-A0SH-01A-11R-A084-07,TCGA-A1-A0SJ-01A-11R-A084-07,TCGA-A1-A0SM-01A-11R-A084-07,TCGA-A1-A0SO-01A-22R-A084-07,TCGA-A2-A04N-01A-11R-A115-07,TCGA-A2-A04P-01A-31R-A034-07,TCGA-A2-A04Q-01A-21R-A034-07,TCGA-A2-A04R-01A-41R-A109-07,⋯,TCGA-OL-A6VQ-01A-12R-A41B-07,TCGA-OL-A6VR-01A-32R-A33J-07,TCGA-OL-A97C-01A-32R-A41B-07,TCGA-PE-A5DD-01A-12R-A27Q-07,TCGA-S3-A6ZF-01A-32R-A32P-07,TCGA-S3-A6ZH-01A-22R-A32P-07,TCGA-S3-AA0Z-01A-11R-A41B-07,TCGA-S3-AA11-01A-31R-A41B-07,TCGA-S3-AA12-01A-11R-A41B-07,TCGA-S3-AA17-01A-11R-A41B-07
1/2-SBSRNA4,6.785132,6.531384,6.815788,6.745408,6.439003,8.190983,6.891659,7.552074,7.141227,6.877849,⋯,6.889313,7.185057,6.608197,7.452912,7.063000,6.480608,7.063943,7.416674,7.555391,7.052109
A1BG,10.061515,9.294596,9.380463,9.621012,9.975146,9.998031,9.273468,8.583678,8.619552,9.536823,⋯,9.751045,9.893515,9.914143,10.722947,9.754787,9.246278,8.388929,8.777843,9.156958,9.338969
A1BG-AS1,7.085018,7.406543,7.455178,7.320522,7.561410,7.930063,7.502942,7.053367,7.294033,7.480218,⋯,8.093161,7.867445,8.054803,8.366981,7.967132,7.500209,8.099842,7.582566,7.531159,7.479362
A1CF,6.987097,5.720335,5.896989,5.890220,6.231723,5.945125,5.944562,6.286815,5.825576,6.194233,⋯,5.522312,5.851550,5.808880,5.933755,6.076932,6.082290,6.215390,5.989860,5.522312,5.978264
A2LD1,9.777167,8.283054,9.019827,8.998635,8.642984,8.774030,8.888976,8.305763,8.756667,8.316350,⋯,8.484823,9.571121,8.143429,8.194549,8.748100,8.796902,7.950549,8.615055,8.654761,8.338903
A2M,13.307465,15.454550,15.398734,15.993144,14.707390,13.692799,14.769829,14.655238,15.173164,14.498198,⋯,15.625608,14.025723,15.369823,15.354516,14.438367,14.278047,14.572121,14.608968,13.217947,15.099874


In [19]:
vsd.data <- cbind(vsd.data, her2.fish[, c('her2_fish_status', 'her2_status_by_ihc')])

write.table(vsd.data, file = '../data/vsd_FeatureCounts_fish.tsv', quote = F, sep = '\t', col.names = NA)

In [20]:
head(as.data.frame(read_tsv("../data/vsd_FeatureCounts_fish.tsv")))

Warning message:
“Missing column names filled in: 'X1' [1]”Parsed with column specification:
cols(
  .default = col_double(),
  X1 = col_character(),
  her2_fish_status = col_character(),
  her2_status_by_ihc = col_character()
)
See spec(...) for full column specifications.
ERROR while rich displaying an object: Error in sprintf(wrap, header, body): 'fmt' length exceeds maximal format length 8192

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     rpr <- mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     rpr <- mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. mime2re

X1,1/2-SBSRNA4,A1BG,A1BG-AS1,A1CF,A2LD1,A2M,A2ML1,A2MP1,A4GALT,⋯,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,tAKR,her2_fish_status,her2_status_by_ihc
TCGA-5T-A9QA-01A-11R-A41B-07,6.785132,10.061515,7.085018,6.987097,9.777167,13.30747,5.788017,5.788017,10.415821,⋯,9.850034,11.37706,6.113148,10.13970,13.33042,11.79847,10.53930,5.522312,Negative,Equivocal
TCGA-A1-A0SE-01A-11R-A084-07,6.531384,9.294596,7.406543,5.720335,8.283054,15.45455,6.174041,5.917433,9.769228,⋯,10.843467,11.52761,8.079875,11.43663,13.08882,11.40790,11.61479,5.522312,Negative,Negative
TCGA-A1-A0SH-01A-11R-A084-07,6.815788,9.380463,7.455178,5.896989,9.019827,15.39873,6.427729,5.828520,10.194560,⋯,10.655458,11.31019,7.339953,12.04475,14.05275,12.00772,11.62560,5.828520,Negative,Equivocal
TCGA-A1-A0SJ-01A-11R-A084-07,6.745408,9.621012,7.320522,5.890220,8.998635,15.99314,6.252312,5.735107,11.035038,⋯,12.831097,11.31361,8.034349,11.69209,13.13071,11.51276,11.12970,5.522312,Negative,Equivocal
TCGA-A1-A0SM-01A-11R-A084-07,6.439003,9.975146,7.561410,6.231723,8.642984,14.70739,6.080947,5.918552,10.028031,⋯,10.584766,11.59618,7.885677,12.06625,13.73885,11.81002,12.09559,5.720899,Positive,Positive
TCGA-A1-A0SO-01A-22R-A084-07,8.190983,9.998031,7.930063,5.945125,8.774030,13.69280,7.605754,6.819967,7.724854,⋯,10.681967,10.32446,8.900671,12.50030,12.11023,11.41151,11.51981,6.317321,Negative,Equivocal
TCGA-A2-A04N-01A-11R-A115-07,6.891659,9.273468,7.502942,5.944562,8.888976,14.76983,6.218269,6.038550,11.057349,⋯,10.012084,11.59105,7.841895,11.12938,12.89119,11.62270,11.31322,5.522312,Negative,[Not Evaluated]
TCGA-A2-A04P-01A-31R-A034-07,7.552074,8.583678,7.053367,6.286815,8.305763,14.65524,12.226380,6.117222,8.988303,⋯,9.169251,11.32724,9.511260,11.00702,13.61124,11.05978,10.94646,6.665229,Negative,[Not Evaluated]
TCGA-A2-A04Q-01A-21R-A034-07,7.141227,8.619552,7.294033,5.825576,8.756667,15.17316,12.794722,6.045674,10.484505,⋯,9.761764,11.39394,8.973027,11.07603,13.65150,11.36913,10.74076,6.045674,Negative,Equivocal
TCGA-A2-A04R-01A-41R-A109-07,6.877849,9.536823,7.480218,6.194233,8.316350,14.49820,6.160318,6.044639,10.032233,⋯,13.160576,12.01775,8.553045,11.32981,13.21611,11.06982,10.50353,5.824972,Negative,Equivocal


In [21]:
nq.fc <- normalize.quantiles(fishCount)

colnames(nq.fc) <- colnames(fishCount)
row.names(nq.fc) <- row.names(fishCount)

nq.fc <- as.data.frame(t(nq.fc))

In [22]:
nq.data <- cbind(nq.fc, her2.fish[, c('her2_fish_status', 'her2_status_by_ihc')])

In [23]:
write.table(nq.data, "../data/norm_quan_FeatureCount_fish.tsv", sep = '\t', quote = F, col.names = NA)

In [25]:
fishCountScore <- as.matrix(fishCountScore)

head(fishCountScore)
str(fishCountScore)

dsq.fc <- DESeqDataSetFromMatrix(countData = fishCountScore, colData = her2.fish.score, design = ~ her2_fish_status)

vsd.fc <- vst(dsq.fc)

head(as.data.frame(assay(vsd.fc)))

vsd.data <- as.data.frame(t(assay(vsd.fc)))

,TCGA-A1-A0SM-01A-11R-A084-07,TCGA-A2-A04X-01A-21R-A034-07,TCGA-A2-A0D0-01A-11R-A00Z-07,TCGA-A2-A0D1-01A-11R-A034-07,TCGA-A2-A0D2-01A-21R-A034-07,TCGA-A2-A0EQ-01A-11R-A034-07,TCGA-A2-A0EU-01A-22R-A056-07,TCGA-A2-A0EV-01A-11R-A034-07,TCGA-A2-A0EW-01A-21R-A115-07,TCGA-A2-A0EY-01A-11R-A034-07,⋯,TCGA-AO-A0JE-01A-11R-A056-07,TCGA-AO-A0JM-01A-21R-A056-07,TCGA-AO-A125-01A-11R-A10J-07,TCGA-AO-A128-01A-11R-A10J-07,TCGA-AO-A129-01A-21R-A10J-07,TCGA-AO-A12C-01A-11R-A10J-07,TCGA-AO-A12D-01A-11R-A115-07,TCGA-AR-A2LJ-01A-12R-A19W-07,TCGA-BH-A18R-01A-11R-A12D-07,TCGA-LL-A5YL-01A-12R-A29R-07
1/2-SBSRNA4,22,58,20,66,45,25,56,28,35,32,⋯,28,19,126,49,30,23,18,27,17,65
A1BG,1051,123,50,262,63,64,422,165,511,291,⋯,434,311,1568,699,245,1769,57,1791,281,1532
A1BG-AS1,124,47,8,98,21,28,133,159,111,132,⋯,134,76,172,203,36,208,24,107,155,69
A1CF,13,2,3,15,2,19,3,4,2,30,⋯,17,12,2,5,1,2,12,2,20,3
A2LD1,359,210,81,87,369,442,446,242,311,571,⋯,284,155,236,623,86,197,265,618,231,454
A2M,30567,28201,302874,15321,114300,29686,22734,30211,67373,23091,⋯,34854,17823,19307,38686,42518,44375,30127,54986,26339,21122


 int [1:23368, 1:44] 22 1051 124 13 359 30567 8 4 1094 4 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:23368] "1/2-SBSRNA4" "A1BG" "A1BG-AS1" "A1CF" ...
  ..$ : chr [1:44] "TCGA-A1-A0SM-01A-11R-A084-07" "TCGA-A2-A04X-01A-21R-A034-07" "TCGA-A2-A0D0-01A-11R-A00Z-07" "TCGA-A2-A0D1-01A-11R-A034-07" ...


factor levels were dropped which had no samples


,TCGA-A1-A0SM-01A-11R-A084-07,TCGA-A2-A04X-01A-21R-A034-07,TCGA-A2-A0D0-01A-11R-A00Z-07,TCGA-A2-A0D1-01A-11R-A034-07,TCGA-A2-A0D2-01A-21R-A034-07,TCGA-A2-A0EQ-01A-11R-A034-07,TCGA-A2-A0EU-01A-22R-A056-07,TCGA-A2-A0EV-01A-11R-A034-07,TCGA-A2-A0EW-01A-21R-A115-07,TCGA-A2-A0EY-01A-11R-A034-07,⋯,TCGA-AO-A0JE-01A-11R-A056-07,TCGA-AO-A0JM-01A-21R-A056-07,TCGA-AO-A125-01A-11R-A10J-07,TCGA-AO-A128-01A-11R-A10J-07,TCGA-AO-A129-01A-21R-A10J-07,TCGA-AO-A12C-01A-11R-A10J-07,TCGA-AO-A12D-01A-11R-A115-07,TCGA-AR-A2LJ-01A-12R-A19W-07,TCGA-BH-A18R-01A-11R-A12D-07,TCGA-LL-A5YL-01A-12R-A29R-07
1/2-SBSRNA4,5.721588,6.463309,5.891367,6.955213,6.378697,5.936117,6.553359,5.908562,6.045605,6.016402,⋯,5.979875,5.994285,7.391755,6.151946,5.994033,5.898654,5.822462,5.987251,5.521542,6.775472
A1BG,9.868031,7.200539,6.627792,8.566231,6.681312,6.711450,8.858064,7.548379,8.916028,8.254257,⋯,8.875347,8.973974,10.674352,9.071733,8.082105,10.915903,6.753658,10.886077,8.011474,10.744206
A1BG-AS1,7.178549,6.284370,5.373267,7.377147,5.808416,6.016723,7.436248,7.506039,7.079153,7.319029,⋯,7.427901,7.251807,7.749722,7.526505,6.129469,8.039931,6.020838,7.231058,7.318505,6.833968
A1CF,5.433395,4.828911,5.009515,5.742510,4.862036,5.754997,4.955385,5.002674,4.826558,5.970082,⋯,5.656866,5.692111,4.860734,5.001746,4.719595,4.872058,5.578955,4.864573,5.610157,4.979072
A2LD1,8.435446,7.808344,7.107184,7.246137,8.708949,8.967105,8.931004,8.002235,8.275296,9.129586,⋯,8.327769,8.081213,8.132838,8.919030,6.902560,7.973573,8.508222,9.410311,7.775484,9.070433
A2M,14.665780,14.590894,18.489788,14.271671,16.833183,14.912751,14.476165,14.717283,15.829201,14.357954,⋯,15.070213,14.691653,14.261631,14.747397,15.285729,15.533844,15.164261,15.794615,14.315495,14.495952


In [26]:
vsd.data <- cbind(vsd.data, her2.fish.score[, c('her2_fish_status', 'her2_status_by_ihc')])

write.table(vsd.data, file = '../data/vsd_FeatureCounts_fish_score.tsv', quote = F, sep = '\t', col.names = NA)

In [ ]:
head(as.data.frame(read_tsv("../data/vsd_FeatureCounts_fish_score.tsv")))

In [28]:
nq.fc <- normalize.quantiles(fishCountScore)

colnames(nq.fc) <- colnames(fishCountScore)
row.names(nq.fc) <- row.names(fishCountScore)

nq.fc <- as.data.frame(t(nq.fc))

In [29]:
nq.data <- cbind(nq.fc, her2.fish.score[, 'her2_fish_status'])

In [30]:
write.table(nq.data, "../data/norm_quan_FeatureCount_fish_score.tsv", sep = '\t', quote = F, col.names = NA)